Original NotoBook: 

https://www.kaggle.com/code/richolson/smiles-rdkit-lgbm-ftw

# SMILES->RDKIT->LGBM->FTW  🧪⚡🚀

**SMILES** *(Simplified Molecular Input Line Entry System)*  
**RDKIT** *(Open-source cheminformatics toolkit)*   
**LGBM** *(Light Gradient Boosting Machine)*  
**FTW** *(For The Win!)*  

## 1. Use RDKit to calculate *chemical descriptors* from our SMILES molecule data
- **Structural Counts:** Ring counts, rotatable bonds, molecular weight
- **Calculated Properties:** LogP (oiliness), TPSA (surface area), qed (drug-likeness), complexity/shape stuff
- We infer these for both **train** and **test** data
- **We are using RDKit to do feature engineering**

## 2. Train models using those features to predict our *targets*:
- **Tg** - Glass transition temperature (°C)
- **FFV** - Fractional free volume
- **Tc** - Thermal conductivity (W/m·K)
- **Density** - Polymer density (g/cm³)
- **Rg** - Radius of gyration (Å)

## 3. Estimate LB Score
- We import a copy of the competition metric: https://www.kaggle.com/code/metric/open-polymer-2025
- Use it to make a guess at how we will rank on the LB
- Likely a bit optimistic since we choose boosting round (epoch) based on best score

## We train unique LGBM models for each target!
- Actually 5 models per target using CV / averaging predictions (**25 models total!**)
- **RDKit is doing the heavy-lifting here** - we just train an LGBM model to figure out how to translate the data to our targets...

### *Friendly Reminder:* If re-using large parts of this work in a public notebook - **please credit where you found the code**

## *Now with extra training data for Tg and Tc!*

Out of the 5 targets - only FFV is well represented in the training data.

We use these datasets to help fill things in for Tg and Tc:
* https://www.kaggle.com/datasets/minatoyukinaxlisa/smiles-tg
* https://www.kaggle.com/datasets/minatoyukinaxlisa/tc-smiles

Source information:
* https://github.com/Jiaxin-Xu/POINT2/blob/main/data/labeled/polyinfo/Tg_SMILES_class_pid_polyinfo_median.csv
* https://github.com/Jiaxin-Xu/POINT2/blob/main/data/labeled/nd/TC_MD_20240306.xlsx
* https://www.kaggle.com/competitions/neurips-open-polymer-prediction-2025/discussion/585178

# Install RDKit
* https://www.kaggle.com/datasets/richolson/rdkit-install-whl

In [1]:
# install RDKit for offline use
!pip install /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/rdkit-install-whl/rdkit_wheel/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore')

# Simple SMILES / RDKit Demo
* So you can see how this works...

In [3]:
molecules = [
    ('CCO', 'Ethanol - simple alcohol'),
    ('CCCCCCCC', 'Octane - long chain'),
    ('c1ccccc1', 'Benzene - aromatic ring'),
]

for smiles, description in molecules:
    mol = Chem.MolFromSmiles(smiles)
    
    print(f"\n{description}")
    print(f"SMILES: {smiles}")
    print(f"  Molecular Weight: {Descriptors.MolWt(mol):.1f}")
    print(f"  LogP (oiliness): {Descriptors.MolLogP(mol):.2f}")
    print(f"  Rotatable Bonds: {Descriptors.NumRotatableBonds(mol)}")
    print(f"  Aromatic Rings: {Descriptors.NumAromaticRings(mol)}")
    print(f"  Complexity (BertzCT): {Descriptors.BertzCT(mol):.0f}")


Ethanol - simple alcohol
SMILES: CCO
  Molecular Weight: 46.1
  LogP (oiliness): -0.00
  Rotatable Bonds: 0
  Aromatic Rings: 0
  Complexity (BertzCT): 3

Octane - long chain
SMILES: CCCCCCCC
  Molecular Weight: 114.2
  LogP (oiliness): 3.37
  Rotatable Bonds: 5
  Aromatic Rings: 0
  Complexity (BertzCT): 25

Benzene - aromatic ring
SMILES: c1ccccc1
  Molecular Weight: 78.1
  LogP (oiliness): 1.69
  Rotatable Bonds: 0
  Aromatic Rings: 1
  Complexity (BertzCT): 72


# Load Train Data

In [4]:
# Load data
comp_train_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
comp_train_df.head(3)

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN


# Define Targets / Check Counts
* Only FFV is well represented in train...

In [5]:
# Define all target properties
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

# Count of non-NaN values for each target column
comp_train_df[targets].count()

Tg          511
FFV        7030
Tc          737
Density     613
Rg          614
dtype: int64

# Look at our additional Tg and Tc data

In [6]:
extra_tg_df = pd.read_csv('/kaggle/input/smiles-tg/Tg_SMILES_class_pid_polyinfo_median.csv')
display(extra_tg_df.head(3))

extra_tc_df = pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv')
display(extra_tc_df.head(3))

,SMILES,PID,Polymer Class,Tg
0,*C*,P010001,Polyolefins,-54.0
1,*CC(*)C,P010002,Polyolefins,-3.0
2,*CC(*)CC,P010003,Polyolefins,-24.1


,TC_mean,SMILES
0,0.244500,*CC(*)C
1,0.225333,*CC(*)CC
2,0.246333,*CC(*)CCC


# Merge Tg and Tc Data with Competition Train Data
* Needs a little reformatting...
* Look at the target counts - that's more like it!

In [7]:
# Prepare extra_tg_df
extra_tg_clean = extra_tg_df[['SMILES', 'PID', 'Tg']].rename(columns={'PID': 'id'})
extra_tg_clean[['FFV', 'Tc', 'Density', 'Rg']] = float('nan')

# Prepare extra_tc_df  
extra_tc_clean = extra_tc_df[['SMILES', 'TC_mean']].rename(columns={'TC_mean': 'Tc'})
extra_tc_clean['id'] = range(len(comp_train_df) + len(extra_tg_df), len(comp_train_df) + len(extra_tg_df) + len(extra_tc_df))
extra_tc_clean[['Tg', 'FFV', 'Density', 'Rg']] = float('nan')

# Reorder columns to match train_df
extra_tg_clean = extra_tg_clean[['id', 'SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']]
extra_tc_clean = extra_tc_clean[['id', 'SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']]

# Combine all datasets into train_df
train_df = pd.concat([comp_train_df, extra_tg_clean, extra_tc_clean], ignore_index=True)

print(train_df[targets].count())

Tg         7719
FFV        7030
Tc         1611
Density     613
Rg          614
dtype: int64


# Define molecular descriptions to be generated by RDKit
* These are properties that RDKit can determine based on SMILES data
* Auto-discovers 400 descriptors defined by RDKit
* Only uses a subset of 192 AUTOCORR2D descriptors (as defined by max_autocorr) 
* We develop models to take this information and predict our actual targets  ('Tg', 'FFV', 'Tc', 'Density', 'Rg')

In [8]:
def get_molecular_descriptors(max_autocorr=10):
    """Get molecular descriptors - either hardcoded list or auto-discovered"""

    descriptor_list_all = []
    test_mol = Chem.MolFromSmiles('CCO')

    # Collect all valid descriptors first
    for name in dir(Descriptors):
        if not name.startswith('_'):
            try:
                func = getattr(Descriptors, name)
                if callable(func):
                    result = func(test_mol)
                    if isinstance(result, (int, float)) and not np.isnan(result):
                        descriptor_list_all.append((name, func))
            except:
                pass

    print(f"🔍 Total discovered descriptors before filtering: {len(descriptor_list_all)}")

    # Sort AUTOCORR2D descriptors by their numeric suffix
    autocorr_descriptors = [
        (name, func)
        for name, func in descriptor_list_all
        if name.startswith('AUTOCORR2D_')
    ]
    autocorr_descriptors.sort(key=lambda x: int(x[0].split('_')[-1]))

    # Select only the lowest-numbered ones
    limited_autocorr = autocorr_descriptors[:max_autocorr]

    # Include all other descriptors
    other_descriptors = [
        (name, func)
        for name, func in descriptor_list_all
        if not name.startswith('AUTOCORR2D_')
    ]

    # Final descriptor list
    descriptor_list = limited_autocorr + other_descriptors

    print(f"✅ Auto-discovered {len(descriptor_list)} descriptors (limited to {max_autocorr} AUTOCORR2D):")
    names = [name for name, _ in descriptor_list]
    print("  " + ", ".join(names))

    feature_names = [name for name, _ in descriptor_list]
    return descriptor_list, feature_names

molecular_descriptors =  get_molecular_descriptors(max_autocorr=10) 

🔍 Total discovered descriptors before filtering: 400
✅ Auto-discovered 218 descriptors (limited to 10 AUTOCORR2D):
  AUTOCORR2D_1, AUTOCORR2D_2, AUTOCORR2D_3, AUTOCORR2D_4, AUTOCORR2D_5, AUTOCORR2D_6, AUTOCORR2D_7, AUTOCORR2D_8, AUTOCORR2D_9, AUTOCORR2D_10, BCUT2D_CHGHI, BCUT2D_CHGLO, BCUT2D_LOGPHI, BCUT2D_LOGPLOW, BCUT2D_MRHI, BCUT2D_MRLOW, BCUT2D_MWHI, BCUT2D_MWLOW, BalabanJ, BertzCT, Chi0, Chi0n, Chi0v, Chi1, Chi1n, Chi1v, Chi2n, Chi2v, Chi3n, Chi3v, Chi4n, Chi4v, EState_VSA1, EState_VSA10, EState_VSA11, EState_VSA2, EState_VSA3, EState_VSA4, EState_VSA5, EState_VSA6, EState_VSA7, EState_VSA8, EState_VSA9, ExactMolWt, FpDensityMorgan1, FpDensityMorgan2, FpDensityMorgan3, FractionCSP3, HallKierAlpha, HeavyAtomCount, HeavyAtomMolWt, Ipc, Kappa1, Kappa2, Kappa3, LabuteASA, MaxAbsEStateIndex, MaxAbsPartialCharge, MaxEStateIndex, MaxPartialCharge, MinAbsEStateIndex, MinAbsPartialCharge, MinEStateIndex, MinPartialCharge, MolLogP, MolMR, MolWt, NHOHCount, NOCount, NumAliphaticCarbocycles, 

# Run RDKit on SMILES Train data
* Predicts molecular descriptors we previously defined
* This is time intensive - so we do it once here instead of in training loop
* This function is also used to process Test data later

In [9]:
def smiles_to_features(smiles_list, descriptor_functions):
   """Convert SMILES strings to raw feature matrix"""
   
   features = []
   total = len(smiles_list)
   
   print(f"Processing {total} SMILES...", end="", flush=True)
   
   for i, smiles in enumerate(smiles_list):
       # Progress indicator every 1000 molecules or at milestones
       if i > 0 and (i % 1000 == 0 or i == total - 1):
           print(f" {i+1}/{total}", end="", flush=True)
       
       mol_features = []
       try:
           mol = Chem.MolFromSmiles(smiles)
           if mol is None:
               # Invalid SMILES - fill with NaN
               mol_features = [np.nan] * len(descriptor_functions)
           else:
               # Calculate each descriptor
               for name, func in descriptor_functions:
                   try:
                       value = func(mol)
                       # Handle problematic values
                       if np.isinf(value) or abs(value) > 1e10:
                           value = np.nan
                       mol_features.append(value)
                   except:
                       # Descriptor calculation failed
                       mol_features.append(np.nan)
       except:
           # Complete failure - fill entire row with NaN
           mol_features = [np.nan] * len(descriptor_functions)
       
       features.append(mol_features)
   
   print(" ✅", flush=True)
   return np.array(features, dtype=float)

descriptor_functions, feature_names = molecular_descriptors
X_raw = smiles_to_features(train_df['SMILES'].values, descriptor_functions)    

Processing 16055 SMILES... 1001/16055 2001/16055 3001/16055 4001/16055 5001/16055 6001/16055 7001/16055 8001/16055 9001/16055 10001/16055 11001/16055 12001/16055 13001/16055 14001/16055 15001/16055 16001/16055 16055/16055 ✅


# Clean Train Data
* Just replaces any NaNs with Median for the column
* This function is also used to process Test data later

In [10]:
def clean_features(X):
    """Handle NaN/inf values and impute missing data"""
    # Create a copy to avoid modifying the original
    X_clean = X.copy()
    
    X_clean[np.isinf(X_clean)] = np.nan
    
    # Count and report missing values
    missing = np.isnan(X_clean).sum()
    print(f"🧹 Cleaned {missing:,} missing values ({missing/X_clean.size*100:.1f}%)")
    
    # Median imputation
    for i in range(X_clean.shape[1]):
        col = X_clean[:, i]
        if np.isnan(col).any():
            X_clean[np.isnan(col), i] = np.nanmedian(col) if not np.isnan(np.nanmedian(col)) else 0
    
    return X_clean

X = clean_features(X_raw)

🧹 Cleaned 195,250 missing values (5.6%)


 # Function to Train an LGBM model for a given target
 * Runs RDKit feature generation on SMILES data
 * Creates / trains a model for a specific target ('Tg', 'FFV', 'Tc', 'Density', 'Rg')
 * Uses 5x cross-validation to utilize all training data (5 models per feature)

In [11]:
def train_target_property(X_target, y_target):
    print(f"📊 Training on {len(y_target)} samples ")
    print(f"📈 Target range: {y_target.min():.4f} to {y_target.max():.4f}")
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_target)
    
    # LightGBM parameters
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': 127,           
        'learning_rate': 0.07,       
        'feature_fraction': 0.8,     
        'bagging_fraction': 0.9,     
        'bagging_freq': 1,           # Bag every iteration
        'lambda_l1': 0.1,            # L1 regularization
        'lambda_l2': 0.1,            # L2 regularization
        'min_data_in_leaf': 10,      # Prevent overfitting
        'verbose': -1,
        'random_state': 42
    }
    
    # 5-fold cross-validation
    cv_scores = []
    models = []
    all_val_true = []
    all_val_pred = []
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_target[train_idx], y_target[val_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=2000,
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )
        
        val_pred = model.predict(X_val)
        cv_score = mean_absolute_error(y_val, val_pred)
        cv_scores.append(cv_score)
        models.append(model)
        
        # Store for competition metric calculation
        all_val_true.extend(y_val)
        all_val_pred.extend(val_pred)
        
        print(f"----Fold {fold+1} Complete / MAE = {cv_score:.4f}", flush=True)
    
    cv_mean = np.mean(cv_scores)
    print(f"===CV: {cv_mean:.4f} ± {np.std(cv_scores):.3f}===")
    return models, scaler, cv_mean, all_val_true, all_val_pred

# Training Loop
* Loops through all 5 polymer target properties (Tg, FFV, Tc, Density, Rg)
* Trains LGBM models for each target
* 5x models per target trained using CV = **25 models total!**

In [12]:
# Store trained models and scalers
trained_models = {}
trained_scalers = {}
cv_scores = []

# Store all predictions for final competition score
all_cv_predictions = {}
all_cv_true = {}

# Train each target
for target in targets:
    print(f"Training {target}...")
    
    # Prepare training data for the current target
    mask = train_df[target].notna()
    X_target = X[mask]
    y_target = train_df[target].values[mask]
    
    models, scaler, cv_score, val_true, val_pred = train_target_property(X_target, y_target)
    trained_models[target] = models
    trained_scalers[target] = scaler
    cv_scores.append(cv_score)
    
    # Store for overall competition score
    all_cv_true[target] = val_true
    all_cv_predictions[target] = val_pred
    print()

Training Tg...
📊 Training on 7719 samples 
📈 Target range: -148.0297 to 495.0000
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[372]	valid_0's l1: 28.4018
----Fold 1 Complete / MAE = 28.4018
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[191]	valid_0's l1: 28.1247
----Fold 2 Complete / MAE = 28.1247
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[645]	valid_0's l1: 27.0196
----Fold 3 Complete / MAE = 27.0196
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[408]	valid_0's l1: 26.8925
----Fold 4 Complete / MAE = 26.8925
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[349]	valid_0's l1: 27.4798
----Fold 5 Complete / MAE = 27.4798
===CV: 27.5837 ± 0.595===

Training FFV...
📊 Training on 7030 samples 
📈 Target range: 0.2270 to 0.7771
Training until 

# Estimate LB Score
* Use the competition metric to estimate LB across all targets
* As mentioned earlier - **probably optimistic** since we choose boosting round (epoch) based on best score
* Import https://www.kaggle.com/code/richolson/open-polymer-2025-metric which is a copy of https://www.kaggle.com/code/metric/open-polymer-2025

In [13]:
# Import the competition metric
import open_polymer_2025_metric as metric

# Create DataFrames for final competition score calculation
cv_true_df = pd.DataFrame()
cv_pred_df = pd.DataFrame()

for target in targets:
    # Pad shorter arrays with NULL_FOR_SUBMISSION to make them same length
    max_len = max(len(all_cv_true[t]) for t in targets)
    
    true_padded = list(all_cv_true[target]) + [metric.NULL_FOR_SUBMISSION] * (max_len - len(all_cv_true[target]))
    pred_padded = list(all_cv_predictions[target]) + [metric.NULL_FOR_SUBMISSION] * (max_len - len(all_cv_predictions[target]))
    
    cv_true_df[target] = true_padded
    cv_pred_df[target] = pred_padded

# Add dummy id column
cv_true_df['id'] = range(len(cv_true_df))
cv_pred_df['id'] = range(len(cv_pred_df))

# Calculate individual competition scores
competition_scores = []
for target in targets:
    comp_score = metric.scaling_error(np.array(all_cv_true[target]), np.array(all_cv_predictions[target]), target)
    competition_scores.append(comp_score)

# Calculate overall competition score
estimated_lb_score = metric.score(cv_true_df, cv_pred_df, 'id')

print("=" * 50)
print(f"Trained: {len(targets)} targets × 5 CV folds = {len(targets) * 5} models")
print(f"Average CV MAE across all targets: {np.mean(cv_scores):.4f}")
print(f"Individual competition scores: {[f'{s:.4f}' for s in competition_scores]}")
print(f"🎯 ESTIMATED LB SCORE: {estimated_lb_score:.4f}")
print("=" * 50)

Trained: 5 targets × 5 CV folds = 25 models
Average CV MAE across all targets: 5.9019
Individual competition scores: ['0.0445', '0.0122', '0.0328', '0.0307', '0.0750']
🎯 ESTIMATED LB SCORE: 0.0444


# Function to Predict using trained LGBM models for a given target
* Runs same RDKit feature generation on test SMILES data
* Uses the 5 trained models to predict a specific target
* Averages predictions from all 5 models for final result

In [14]:
def predict_target_property(test_df, target_name, models, scaler):
    
    print(f"PREDICTING: {target_name}")
    
    if models is None or scaler is None:
        print(f"❌ No trained model available for {target_name}, returning zeros")
        return np.zeros(len(test_df))
    
    # Get molecular features - step by step
    descriptor_functions, _ = molecular_descriptors
    X_raw = smiles_to_features(test_df['SMILES'].values, descriptor_functions)
    X = clean_features(X_raw)
    
    # Scale features using same scaler from training
    X_scaled = scaler.transform(X)
    
    # Average predictions from all CV folds
    fold_predictions = []
    for model in models:
        pred = model.predict(X_scaled)
        fold_predictions.append(pred)
    
    predictions = np.mean(fold_predictions, axis=0)
    print(f"📊 Predictions range: {predictions.min():.4f} to {predictions.max():.4f}")
    
    return predictions

# Predict All Targets / Submit
* Predict on test data
* Creates final submission CSV with all predictions

In [15]:
# Load test data
test_df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

print(f"\nMAKING PREDICTIONS...")
all_predictions = {}
for target in targets:
    predictions = predict_target_property(
        test_df, target, trained_models[target], trained_scalers[target]
    )
    all_predictions[target] = predictions

# Create submission
submission = pd.DataFrame({'id': test_df['id']})
for target in targets:
    submission[target] = all_predictions[target]

submission.to_csv('submission.csv', index=False)

print(f"Predicted: {len(test_df)} test samples")
print(f"Saved: submission.csv")

print(f"\n👀 SUBMISSION PREVIEW:")
print(submission.head().to_string(index=False, float_format='%.4f'))


MAKING PREDICTIONS...
PREDICTING: Tg
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.7%)
📊 Predictions range: 140.0810 to 155.1468
PREDICTING: FFV
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.7%)
📊 Predictions range: 0.3504 to 0.3778
PREDICTING: Tc
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.7%)
📊 Predictions range: 0.1683 to 0.2554
PREDICTING: Density
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.7%)
📊 Predictions range: 1.0848 to 1.1699
PREDICTING: Rg
Processing 3 SMILES... 3/3 ✅
🧹 Cleaned 37 missing values (5.7%)
📊 Predictions range: 19.9183 to 20.3570
Predicted: 3 test samples
Saved: submission.csv

👀 SUBMISSION PREVIEW:
        id       Tg    FFV     Tc  Density      Rg
1109053969 153.4488 0.3754 0.1683   1.1699 19.9183
1422188626 155.1468 0.3778 0.2307   1.0848 20.0204
2032016830 140.0810 0.3504 0.2554   1.0999 20.3570


Result:

Score: 0.059

Rank: 66 (2025-0621-1:47, JST)

Your First Entry!
Welcome to the leaderboard!